# Result Collection, Regional population projection

#### may 2022 version
only the output part

### email from Sturla


I want a file with a few different policy relavant population aggregates:
•	Total population 
•	Population in childcare age (1-5 at end of year)  
•	Population in primary school age (age 6-12 at the end of year)  
•	Population in lower secondary school age (age 13-15 at the end of year)  
•	Elderly population (80+ at end of year)
•	Very old population (90+ at end of year)

We should have one observation of the number of people in these groups across all municipalities, years and simulations. Returning a file with variables: knr, year, sim_id, pop_all, pop0105, pop0612, pop1315, pop8000 pop9000 and 10m obs (356*29*1000). 


#### Python scripts to collect and orgainze the simulation results from microsimulations.
Input files:
- sim_outxxx.pkl
generated using sim_main.py, the parameters can be set through sim_para.py.
the python scripts using paralle computational techinque and can be ran on server stata-p3 using python3 command directly from command line. 


In [1]:
### libary importation and some global parameters:

import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as spo
import pandas as pd
import seaborn as sns
import subprocess
import os 
from itertools import compress
import copy
import pickle

pd.set_option('float_format', '{:f}'.format)


In [2]:
import sim_para



# maxage=120
# range_fertility=49-15+1
# max_year=31
# number_region=356
# move_age=69


sim_mark=sim_para.sim_mark

path = sim_para.path
max_year=sim_para.max_year
base_year=sim_para.base_year
move_age=sim_para.move_age
number_region=sim_para.number_region
maxage=sim_para.maxage
range_fertility=sim_para.range_fertility
start_year=sim_para.start_year
png_dir=path+'/gif_dir/'

## simulation results are saved with suffixes 'xxx' so that different 
## simulation runs can be used

# run_index_list=['2', '3', '4']
run_index_list=[str(i) for i in range(sim_para.number_loop)]
#for i in range(4):
#    run_index_list.pop(0)

## select the three regions

region_selection=[0, 22, 122, 227]
region_name=['Oslo','Utsira', 'Eidsvoll', 'Bygland']

Region_label=["county "+str(i)+" " for i in range(1, number_region+1)]
Region_label[122]='Eidsvoll'  
Region_label[0]='Oslo'  
Region_label[22]='Utsira'
Region_label[227]='Bygland'  
Region_label.append("nationwide ")

total number of available CPUs is 32


## input simulation results

In [3]:

res=[]
## read in the simulatin rsults
for index in run_index_list:
    file = open(path+'/sim_out'+sim_mark+index+'.pkl', 'rb')
    res.append(pickle.load(file))
    file.close()



In [4]:
## find out the number of simulations and number of prediction years
number_loop=len(res)
print('simulation number ',number_loop)
number_year=len(res[0][0])-1
print("simulate %s years" % number_year)

simulation number  1000
simulate 29 years


In [5]:
## separate the informations

'''
# the structure of the output
# list 
## first level: index simulation runs, list [number_loop]
## second level: [0] summary of population, list [number_year]
                         3 dimensional array [age, sex, region]
                 [1] summary of immigrants, list [number_year]
                         3 dimensional array [age, sex, region]
                 [2] summary of newborns, 
                         2 dimensional array [sex, region]
                 [3] summary of deaths,     
                         3 dimensional array [age, sex, region]
                 [4] summary of outmigrants,
                         3 dimensional array [age, sex, region]                
                 [5] summary of in-country mover,
                         4 dimensional array [age, sex, region, 2]
We reorganize this to 5 listst ,which correspond to the above 5 different information and have the same structure
'''
sim_size_full=[]
sim_im=[]
sim_newborn=[]
sim_dead=[]
sim_out=[]
sim_move=[]
sim_totalnewborn=[]
sim_fertile=[]
sim_fertile_actual=[]

for i in range(len(res)):
    sim_size_full.append(res[i][0])
    sim_im.append(res[i][1])
    sim_newborn.append(res[i][2])
    sim_dead.append(res[i][3]) 
    sim_out.append(res[i][4]) 
    sim_move.append(res[i][5])
    sim_fertile.append(res[i][6])
    sim_fertile_actual.append(res[i][7])
    sim_totalnewborn.append([np.sum(res[i][2][j]) for j in range(number_year+1)])

len(sim_size_full)


1000

In [25]:
import time
start_time=time.time()
list_out=[]
for i in range(2):
    for j in range(2):
        temp=res[i][0][j+1]
        for r in range(2):
        # loop, projection year, region, total population, childcare,primary, lower, elderly,over90
            list_temp=[i+1, start_year+j, r, np.sum(temp[:,:,r]), np.sum(temp[1:6,:,r]), np.sum(temp[6:13,:,r]),  np.sum(temp[13:16,:,r]), np.sum(temp[80:,:,r]), np.sum(temp[90:,:,r])]
            list_out.append(list_temp)
            print(r, time.time()-start_time)
        

print(time.time()-start_time)
list_out

0 0.00022482872009277344
1 0.00030732154846191406
0 0.00035691261291503906
1 0.00039124488830566406
0 0.00044655799865722656
1 0.0004811286926269531
0 0.0005223751068115234
1 0.0005559921264648438
0.0006256103515625


[[1, 2022, 0, 709070.0, 38340.0, 50629.0, 21441.0, 22229.0, 4808.0],
 [1, 2022, 1, 14871.0, 807.0, 1254.0, 618.0, 691.0, 122.0],
 [1, 2023, 0, 714458.0, 38186.0, 50315.0, 21696.0, 23228.0, 4740.0],
 [1, 2023, 1, 14905.0, 819.0, 1220.0, 627.0, 712.0, 123.0],
 [2, 2022, 0, 708804.0, 38516.0, 50626.0, 21375.0, 22256.0, 4795.0],
 [2, 2022, 1, 14803.0, 799.0, 1255.0, 621.0, 693.0, 126.0],
 [2, 2023, 0, 714811.0, 38617.0, 50298.0, 21593.0, 23214.0, 4736.0],
 [2, 2023, 1, 14796.0, 810.0, 1212.0, 624.0, 713.0, 131.0]]

In [16]:
list_out

[[1, 2022, 0, 709070.0, 38340.0, 50629.0, 21441.0, 22229.0, 4808.0],
 [1, 2022, 1, 14871.0, 807.0, 1254.0, 618.0, 691.0, 122.0],
 [1, 2023, 0, 714458.0, 38186.0, 50315.0, 21696.0, 23228.0, 4740.0],
 [1, 2023, 1, 14905.0, 819.0, 1220.0, 627.0, 712.0, 123.0],
 [2, 2022, 0, 708804.0, 38516.0, 50626.0, 21375.0, 22256.0, 4795.0],
 [2, 2022, 1, 14803.0, 799.0, 1255.0, 621.0, 693.0, 126.0],
 [2, 2023, 0, 714811.0, 38617.0, 50298.0, 21593.0, 23214.0, 4736.0],
 [2, 2023, 1, 14796.0, 810.0, 1212.0, 624.0, 713.0, 131.0]]

In [21]:
test_out=pd.DataFrame(list_out)

In [22]:
df_knr=pd.read_csv(path+'/'+'df_knr.csv')
test_out = test_out.set_axis(["loop", "year", "region", "population", "childcare", "primary", "lower", "elderly", "over90"], axis=1)
df_uncertainty=test_out.merge(df_knr, on='region')
df_uncertainty.drop(columns=['region'], inplace=True)

In [24]:
df_uncertainty.to_csv(path+'/'+'uncertainty_sim.csv', index=False)

In [ ]:

temp_sim_size_full=np.sum(sim_size_full, 0)/number_loop
temp_sim_dead=np.sum(sim_dead, 0)/number_loop
temp_sim_im=np.sum(sim_im, 0)/number_loop
temp_sim_out=np.sum(sim_out, 0)/number_loop
temp_sim_newborn=np.sum(sim_newborn, 0)/number_loop
temp_sim_move=np.sum(sim_move, 0)/number_loop

res_sim_size_full=np.zeros((number_year+1,120,2,356))
res_sim_dead=np.zeros((number_year+1,120,2,356))
res_sim_newborn=np.zeros((number_year+1,2,356))
res_sim_im=np.zeros((number_year+1,120,2,356))
res_sim_out=np.zeros((number_year+1,120,2,356))
res_sim_move=np.zeros((number_year+1,120,2,356,2))

                    
for i in range(number_year+1):
    #print(i)
    for j in range(120):
        for k in range(2):
            for l in range(356):
                res_sim_size_full[i,j,k,l]=temp_sim_size_full[i][j,k,l]
                res_sim_dead[i,j,k,l]=temp_sim_dead[i][j,k,l]
                if j==0:
                    res_sim_newborn[i,k,l]=temp_sim_newborn[i][k,l]
                if j<=move_age:
                    res_sim_im[i,j,k,l]=temp_sim_im[i][j,k,l]
                    res_sim_out[i,j,k,l]=temp_sim_out[i][j,k,l]
                    # moving out of region l
                    res_sim_move[i,j,k,l,0]=temp_sim_move[i,j,k,l,0]
                    res_sim_move[i,j,k,l,1]=temp_sim_move[i,j,k,l,1]


In [ ]:
## generate a array with number of total population at year 25 for selected regions.
#
pop_size_selected=np.zeros((number_loop, len(region_selection)))
p_year=number_year
for j in range(len(region_selection)):
    pop_size_selected[:,j]=[np.sum([sim_size_full[i][p_year][:,:,region_selection[j]]]) for i in range(number_loop)]

In [ ]:
pd.DataFrame(pop_size_selected).to_csv(path+"/pop_region.csv")
print("done")

In [ ]:
print(sim_totalnewborn[:2])
out=np.zeros((number_loop, number_year+1))
for j in range(number_year+1):
    out[:,j]=[sim_totalnewborn[i][j] for i in range(number_loop)]


In [ ]:
fertility_table=[np.zeros((number_region,range_fertility)) for i in range(max_year)]

def set_fertility(file):
    datafile=path+'/'+file
    data_in=pd.read_csv(datafile)  
        #print(data_in)
    for index, row in data_in.iterrows():
        # index_temp starts at zero
        index_temp=int(row.year-base_year-1)
        # region: 0-355
        r=int(row.region)
        #age starts from 15
        a=int(row.age)-15
        fertility_table[index_temp][r,a]=row.prob 
        
set_fertility("fertility.csv")
        
# calculate teh expected number of birth
count=np.zeros((number_loop,number_year))
birth=np.zeros((number_loop,number_year))

for ii in range(number_loop):
    for i in range(number_year):
        year=base_year+i+1
        for r in range(number_region):
            for j in range(range_fertility):
                # take the number of females (aged 14 to 48 at the end of last year) 
                a=14+j
                birth[ii,i]=birth[ii,i]+sim_size_full[ii][i][a,1,r]*fertility_table[i][r,j]
                #birth[ii,i]=birth[ii,i]+sim_size_full[ii][i][a,1,r]*0.05
                count[ii,i]=count[ii,i]+sim_size_full[ii][i][a,1,r]

# the simulated birth
for i in range(number_year):
    #number of total fertile women
    print("year ", start_year+i)
    print("average number of fertile women", np.sum(count[:,i])/number_loop)
    a=np.sum(birth[:,i])/number_loop
    b=np.sum(out[:,i+1])/number_loop
    diff=a-b
    print(a, b, diff)
    for j in range(10):
        start=j*100
        stop=(j+1)*100
        a=np.sum(birth[start:stop,i])/100
        b=np.sum(out[start:stop,i+1])/100
        diff=a-b
        print(j, a, b, diff)
            

In [ ]:
# make the array into a dataframe, where there are (number_year+1)*121*2*356 lines
# where columns are 'year_projection', 'age', 'gender', 'region', 'population'

lines=(number_year+1)*121*2*356

c=0
sim_output=np.zeros([lines,5])
for i in range(number_year+1):
    for j in range(120):
        for k in range(2):
            for l in range(356):
                sim_output[c,0]=i
                sim_output[c,1]=j
                sim_output[c,2]=k
                sim_output[c,3]=l
                sim_output[c,4]=res_sim_size_full[i,j,k,l]
                c += 1
df_sim_size=pd.DataFrame(sim_output)
df_sim_size.columns=['year_projection', 'age', 'gender', 'region', 'population']

# look at the projection for year 15 (year=0 is the base year)
df_sim_size[df_sim_size['year_projection']==15].head()

In [ ]:
df_sim_size[df_sim_size['age']==120].sum()

In [20]:
# ## get the knr variable into the dataframe 
# and save to output file

datafile=path+'/'+'population.csv'
data_population=pd.read_csv(datafile)

a=data_population['knr'].unique()
a.sort()

df_knr=pd.DataFrame(a)

df_knr.reset_index(inplace=True)

df_knr.head()

df_knr.columns=['region', 'knr' ]

df_knr.to_csv(path+'/'+'df_knr.csv', index=False)

In [ ]:


df_knr=pd.read_csv(path+'/'+'df_knr.csv')

print(df_knr.head())

## merge the region variable in

df_pop=df_sim_size.merge(df_knr, on='region')

df_pop.drop(columns=['region'], inplace=True)

# df_pop[(df_pop['year_projection']==15) & (df_pop['age']==55)].tail()

df_pop.to_csv(path+'/'+'population_sim.csv', index=False)

In [ ]:
## check cohort 
df_pop['year']=df_pop['year_projection']+base_year+1
df_pop['cohort']=df_pop['year']-df_pop['age']

df_oslo=df_pop[(df_pop['knr']==301) & (df_pop['cohort']==2010)].groupby('age', as_index=False).sum()
df_oslo.head()

# plot the number of the 2010 cohort at Oslo

plt.figure(figsize=(13,10), dpi= 80)
plt.plot(df_oslo['age'], df_oslo['population'], '-', color='gray')
plt.show()

Components by municipality and year (6outcomes x 356munic x 30years)
-	Births
-	Deaths
-	Immigrations
-	Emigrations
-	In-migrations (domestic)
-	Out-migration (domestic)


In [ ]:
## imput the size of emigrants and out-emigrants
im_size=np.zeros(31)
em_size=np.zeros(31)

def set_im_size(file):
    datafile=path+'/'+file
    data_in=pd.read_csv(datafile)
    for index, row in data_in.iterrows():
        index_temp=int(row.year-base_year-1)       
        im_size[index_temp]=(row.tot_immigrants)
        em_size[index_temp]=(row.tot_emigrants)

set_im_size("tot_migration.csv")

In [ ]:
# res_sim_size_full[time_index,age,gender,region]
print(res_sim_size_full[0,:,:,:].sum())
print(res_sim_size_full[0,4,0,0])

In [ ]:
lines=(number_year+1)*121*2*356

c=0
sim_out_2=np.zeros([lines,10])

for i in range(number_year+1):
    #print(i)
    for j in range(120):
        for k in range(2):
            for l in range(356):
                sim_out_2[c,0]=i
                sim_out_2[c,1]=j
                sim_out_2[c,2]=k
                sim_out_2[c,3]=l
                sim_out_2[c,6]=res_sim_dead[i,j,k,l]
                if j==0:
                    sim_out_2[c,5]=res_sim_newborn[i,k,l]
                if j<= move_age:
                    sim_out_2[c,4]=res_sim_im[i,j,k,l]
                    sim_out_2[c,7]=res_sim_out[i,j,k,l]
                    sim_out_2[c,8]=np.sum(res_sim_move[i,j,k,l,0])
                    sim_out_2[c,9]=np.sum(res_sim_move[i,j,k,l,1])
                c += 1
df_sim_size_2=pd.DataFrame(sim_out_2)
df_sim_size_2.columns=['year_projection', 'age', 'sex', 'region', 'immigrants', 'births', 'deaths', 'outmigrants', 'move_out', 'move_in']
df_sim_size_2.head()


In [ ]:

## nationwide outmigration by project year
print('year  simulated size   given size')
for i in range(number_year):
    print(i+start_year, np.sum(res_sim_out[i+1,:,:,:]), em_size[i])


In [ ]:
df_pop=df_sim_size_2.merge(df_knr, on='region')

df_pop.drop(columns=['region'], inplace=True)

# check consistency between move in/out
print(df_pop[df_pop['year_projection']==1]['move_out'].sum())
print(df_pop[df_pop['year_projection']==1]['move_in'].sum())

df_pop.to_csv(path+'/'+'components_sim.csv', index=False)

## Output File requested by Sturla for the report

I want a file with a few different policy relavant population aggregates:
 - 	Total population 
 - 	Population in childcare age (1-5 at end of year)  
 - 	Population in primary school age (age 6-12 at the end of year)  
 - 	Population in lower secondary school age (age 13-15 at the end of year)  
 - 	Elderly population (80+ at end of year)
 - 	Very old population (90+ at end of year)

We should have one observation of the number of people in these groups across all municipalities, years and simulations. Returning a file with variables: knr, year, sim_id, pop_all, pop0105, pop0612, pop1315, pop8000 pop9000 and 10m obs (356*29*1000). 


All information is based on the number of residents, so we work with sim_size_full

sim_size_full is a list (number_loop) of lists (number_year) of 3 dimensional array [age, sex, region]

#### sim_size_full[i][j][a,s,r]

denotes the number of projected population size for 
- sim loop i 
- projection year [base_year+j]
- of age a, sex s in region r

In [ ]:
list_out=[]
for i in range(1):
    for j in range(1):
        temp=sim_size_full[i][j+1]
        for r in range(1):
        # loop, projection year, region, total population, childcare,primary, lower, elderly, 
        list_temp=[i+1, start_year+j, r, np.sum(temp[:,:,r]), np.sum(temp[1:6,:,r]), np.sum(temp[6:13,:,r]),  np.sum(temp[13:16,:,r]), np.sum(temp[80:,:,r]), np.sum(temp[90:,:,r])]
        list_out.append(list_temp)
        

In [ ]:
len(sim_size_full)